# Script to track all the videos from ant arena experiments  

In this script we build a list of all of the avi files to track. We then run the 2D tracker on each file using the tracker class. This results in a text file of tracked contours saved with the video files.   

In [1]:
import matlab.engine
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import sys


# %qtconsole
%matplotlib auto

import glob
sys.path.append('/home/gravishlab/Documents/Python/')
sys.path.append('/home/gravishlab/Documents/Python/Tracker/')
sys.path.append('/home/gravishlab/Documents/Python/Tracker/Tracker/')
from Tracker.Tracker import Tracker

import multiprocessing

Using matplotlib backend: TkAgg


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
vid_locations = '/media/gravishlab/SeagateExpansionDrive/AntTrack/'

# glob is like dir in matlab, it searches for files
# file_list = glob.glob(os.path.join(vid_locations, '*12-0000.avi'))
#file_list = glob.glob(os.path.join(vid_locations, '4mm_clear/*12-0000.avi'))
# file_list = glob.glob(os.path.join(vid_locations, 'AntTunnel/2mm_clear/*18-0000.avi'))
# file_list = glob.glob(os.path.join(vid_locations, '**/*12-0000.avi'))
file_list = glob.glob(os.path.join(vid_locations, '**/**/*0000.mp4'))
print(len(file_list))
file_list = sorted(file_list)[0:1] #   [::20] #[0:3000:5]                


print('Total Number of Videos: ',len(file_list))

10842
Total Number of Videos:  1


In [3]:
# %qtconsole  

## Build file list for testing

In [4]:

# # this is a list of files that have or have not been processed
# processing_manifest = os.path.join(vid_locations, 'processing_manifest.json')

# if glob.glob(processing_manifest) == []:

#     file_list = []
#     for root, dirs, files in os.walk(vid_locations):
#         for file in files:
#             if(os.path.splitext(file)[1] == '.mp4' and # only mp4 files
#                os.path.basename(file)[0] != '5'):      # dont track video 5
#                 file_list.append([os.path.join(root, file), 0])
                
#     with open(processing_manifest, 'w') as out:
#         json.dump(file_list, out, sort_keys=True, indent=4)
        
# else:
#     with open(processing_manifest, 'r') as infile:
#         file_list = json.load(infile)
    
# len(file_list)

In [5]:

# define the region of interest for tracking
# ROI = (30, 30, 550, 1174)

In [6]:
# this function is called by the parallel tracker below. 

def track_or_skip(filename):

    try:
        # define tracking parameter for green vs. clear substrates
        if 'clear' in filename.split('/')[5]:
            thresh = 0.2
#             print(type(thresh))
        elif 'green' in filename.split('/')[5]:
            thresh = 0.1
        else:
            thresh = 0.2
        
        
        video = Tracker(filename, min_object_size=200, verbose='False') #, CLEAR -- verbose='False'
#         video = Tracker(filename, min_object_size=200, verbose='False') # GREEN, verbose='False'
#         video = Tracker(filename, ROI = ROI)
#       print(video.file_exists, '---',filename)
#        print(video.file_exists)

        if video.file_exists == False:
            #print('in ' , video.file_name)
            
#             video.threshold_val = 0.15 # GREEN - how diff from background (1) will identify an object
            video.threshold_val = thresh # CLEAR - how diff from background (1) will identify an object
            
            video.load_video()
            video.compute_background()          # form background image
            video.remove_background()           # remove background
            video.threshold()                   # threshold to segment features
            video.find_distance()               # takes, dist tranform, finds peaks, associates to ants, finds head/gaster
            video.morpho_closing()
            video.find_objects()
            video.draw_contours()
            print('done?')
            video.save_JSON()

            print('Saved  ---',filename)
        else:
            print('Exists ---',filename)
    except:
        print('error--- ', filename)

## Now farm out tracking to a set of multiprocessing functions

In [7]:
# print(sorted(file_list)[0:10])

In [ ]:
# this does the tracking in parallel. Alternatively this could just be a for loop calling track_or_skip on each file in filename

p = multiprocessing.Pool(1)
p.map(track_or_skip, sorted(file_list)) #[200:205]
print('All Done!')

#filenames = [file for file, done in file_list[0:2]]
#p.map(track_or_skip, filenames)
#print(file_list)


## Check through list and re-run on files that got skipped

In [10]:
not_done = []
for kk, filename in enumerate([file for file in file_list]):
#     print(kk)
    
    if glob.glob(os.path.join(os.path.dirname(filename), 
                              os.path.splitext(os.path.basename(filename))[0]) + '_contours.txt') == []:
        not_done.append(filename)
        
        

In [11]:
## CHECK IF WORKING USING MY TRACKALLVIDEOS file
#for kk, vname in enumerate(sorted(file_list[0:2])): #[129:130]: # total 131
#    print(kk, ' --- ', vname)
#    # Load in images to memory during construction
#    print(vname)
#    video = Tracker(vname,min_object_size=40) # ROI= (30, 30, 550, 1150) , verbose='False'
#    print(video.file_exists)

In [ ]:
#for kk, vname in enumerate(sorted(file_list)[150:153]): #[129:130]: # total 131
#    video = Tracker(vname,min_object_size=40) # ROI= (30, 30, 550, 1150) , verbose='False'
#    
#    video.file_name=os.path.splitext(os.path.basename(video.videoname))[0] + '_contours.txt'
#    video.out_file = os.path.join(video.file_path, video.file_name)
#    
#    print( (glob.glob(video.out_file) != []), '--', video.out_file)
#    print(kk, ' --- ', vname, video.file_exists)

